<a href="https://colab.research.google.com/github/WoogiWooks/NDSS_LAB/blob/main/%ED%95%84%EB%93%9C%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%B2%98%EB%A6%AC%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 스펙트럼 데이터 뽑아내기

In [ ]:
# 총합본!!!!!
# 실행 전 마운트 필요



import pandas as pd
import os
import glob



# 엑셀 파일의 위치 경로설정
# 경로 확인  # 파일이 아닌 폴더경로로
directory = '/content/drive/MyDrive/Field/raw_data/20210819_Field1'
data_list = glob.glob(os.path.join(directory, '*.dat'))

allData = []
for file in data_list:
  df = pd.read_csv(file, sep="	", encoding="utf-16", header=None)
  allData.append(df)
F1 = pd.concat(allData, axis=0,  ignore_index=True)



# 레퍼런스와 스펙데이터 나누기
# 해당 단어가 포함되도록 레퍼런스 이름 설정해줘야
# 레퍼런스 = F_WR
# 스펙 데이터 = F1
F_WR = F1[F1[0].str.contains('white', case=False) | F1[0].str.contains('wr', case=False) | F1[0].str.contains('w', case=False) ]
F_WR = F_WR.reset_index()
F_WR = F_WR.drop(columns='index')

F1 =  F1[~F1[0].str.contains('white', case=False) & ~F1[0].str.contains('wr', case=False) & ~F1[0].str.contains('w', case=False) ]
F1 = F1.reset_index()
F1 = F1.drop(columns='index')



# 데이터 필드포인트별로 각각 나눠주기
n=0          
for i in range(0, F1.shape[0]//2151):
  A=2151*(i+1)
  globals()['F1_{0}'.format(i)] = F1.loc[n:A-1]
  n = A



# 전치행렬로 변환
for i in range(0, F1.shape[0]//2151):
   globals()['F1_{0}'.format(i)] = globals()['F1_{0}'.format(i)].T



# 데이터별 컬럼명 350~2500로 바꿔주기
for i in range(0, F1.shape[0]//2151):
  globals()['F1_{0}'.format(i)].columns = list(range (350, 2501))



# 필드 정보를 column 데이터로 만든 뒤 인덱스로 변환
for i in range(0, F1.shape[0]//2151):
  Inform =  globals()['F1_{0}'.format(i)].iloc[0,0]
  Inform = Inform[11:-4]
  globals()['F1_{0}'.format(i)]["inform"] = Inform
  globals()['F1_{0}'.format(i)] = globals()['F1_{0}'.format(i)].drop(index=[0, 1, 2])
  col1 = globals()['F1_{0}'.format(i)].columns[-1:].to_list()
  col2 = globals()['F1_{0}'.format(i)].columns[:-1].to_list()
  new_col = col1 + col2
  globals()['F1_{0}'.format(i)] = globals()['F1_{0}'.format(i)][new_col]
  globals()['F1_{0}'.format(i)].set_index("inform", inplace=True)



# 데이터들 하나로 병합
Result = pd.concat([F1_0, F1_1])
for i in range(1,(F1.shape[0]//2151)-1):
  Result = pd.concat([Result, globals()['F1_{0}'.format(i+1)]])



# 데이터가 총 몇개인지 Result로 부터 가져옴
Total_Num = Result.shape[0]   
Sep_Num = int((Total_Num)/4)   # 하나 데이터에 4번촬영했으므로 4로 나눔
Sep_Num                                    # 촬영지역 개수



# 묶기
n=0
for i in range (1, Sep_Num+1):
  globals()['F1_new_{0}'.format(i)] = pd.concat([globals()['F1_{0}'.format(n)], globals()['F1_{0}'.format(n+1)], globals()['F1_{0}'.format(n+2)], globals()['F1_{0}'.format(n+3)]])
  n+=4



# 평균치 구해서 데이터에 붙여주기
for i in range(1, Sep_Num+1):
  name = globals()['F1_new_{0}'.format(i)].index[0]
  name = name[0:-1] + "mean"
  
  globals()['F1_new_{0}_mean'.format(i)] = pd.DataFrame(globals()['F1_new_{0}'.format(i)].mean())
  globals()['F1_new_{0}_mean'.format(i)] = globals()['F1_new_{0}_mean'.format(i)].T.rename(index={0:name})
  globals()['F1_new_{0}'.format(i)] = pd.concat([globals()['F1_new_{0}'.format(i)], globals()['F1_new_{0}_mean'.format(i)]])



# 평균치 붙은 데이터들 총 통합
New_Result = pd.concat([F1_new_1, F1_new_2])
for i in range(3, Sep_Num+1):
  New_Result = pd.concat([New_Result, globals()['F1_new_{0}'.format(i)]])



# 저장
# 파일명 설정
new_dr = '/content/drive/MyDrive/Field/prepro_data/20210819&20'
if not os.path.exists(new_dr):
  os.mkdir(new_dr)
with pd.ExcelWriter(os.path.join(new_dr, '20210820_Field2_정리.xlsx')) as writer:
  New_Result.to_excel(writer)

## 레퍼런스 데이터 뽑아내기

In [ ]:
# 데이터 필드포인트별로 각각 나눠주기
n=0
F_WR_list = []
for i in range(0, F_WR.shape[0]//2151):
  A=2151*(i+1)
  globals()['F_WR_{0}'.format(i)] = F_WR.loc[n:A-1]
  F_WR_list.append('F_WR_{0}'.format(i))
  n = A



# 전치행렬로 변환
for i in range(0, F_WR.shape[0]//2151):
  globals()['F_WR_{0}'.format(i)] = globals()['F_WR_{0}'.format(i)].T



# 데이터별 컬럼명 350~2500로 바꿔주기
for i in range(0, F_WR.shape[0]//2151):
  globals()['F_WR_{0}'.format(i)].columns = list(range (350, 2501))



# 필드 정보를 column 데이터로 만든 뒤 인덱스로 변환
for i in range(0, F_WR.shape[0]//2151):
  Inform =  globals()['F_WR_{0}'.format(i)].iloc[0,0]
  Inform = Inform[11:-4]
  globals()['F_WR_{0}'.format(i)]["inform"] = Inform
  globals()['F_WR_{0}'.format(i)] = globals()['F_WR_{0}'.format(i)].drop(index=[0, 1, 2])
  col1 = globals()['F_WR_{0}'.format(i)].columns[-1:].to_list()
  col2 = globals()['F_WR_{0}'.format(i)].columns[:-1].to_list()
  new_col = col1 + col2
  globals()['F_WR_{0}'.format(i)] = globals()['F_WR_{0}'.format(i)][new_col]
  globals()['F_WR_{0}'.format(i)].set_index("inform", inplace=True)



# 데이터들 하나로 병합
if 'F_WR_1' in F_WR_list:
  Result = pd.concat([F_WR_0, F_WR_1])
  for i in range(1,(F_WR.shape[0]//2151)-1):
    Result = pd.concat([Result, globals()['F_WR_{0}'.format(i+1)]])
else: 
  Result = F_WR_0



# 저장
# 파일명 설정
new_dr = '/content/drive/MyDrive/Field/prepro_data/20210819&20'
if not os.path.exists(new_dr):
  os.mkdir(new_dr)
with pd.ExcelWriter(os.path.join(new_dr, '20210820_Field2_WR.xlsx')) as writer:
  Result.to_excel(writer)